In [1]:
import numpy as np
import random

dim = 2 #random.randint(40,100)
xt = np.random.rand(dim)

Generate pseudo-norm

In [2]:
norm = np.random.rand(dim)*0.5+0.5
norm = norm*np.linspace(1.0,1.0*dim,dim)

from array import array
output_file = open('norm.dat', 'wb')
float_array = array('d',norm)
float_array.tofile(output_file)
output_file.close()

input_file = open('norm.dat', 'rb')
read_array = array('d')
read_array.frombytes(input_file.read())

def readVector(filename):
    from array import array
    input_file = open(filename,'rb')
    read_array = array('d')
    read_array.frombytes(input_file.read())
    return np.array(read_array)

pseudo forward-run and adjoint run

In [3]:
forwardFilename = 'forward.txt'
adjointFilename = 'adjoint.txt'
gradientFilename = 'gradient.dat'

def writeScalar(scalar, scalarFilename):
    fID = open(scalarFilename,'w')
    fID.write("{:16E}".format(scalar))
    fID.close()
    return 

def forward(x):
    J = 0.5*sum((x-xt)*norm*(x-xt))
    writeScalar(J,forwardFilename)
    return
    
def adjoint(x, gradientFilename):
    dJ = x-xt
    from array import array
    float_array = array('d',dJ)
    fID = open(gradientFilename, 'wb')
    float_array.tofile(fID)
    fID.close()
    
    writeScalar(sum((x-xt)*norm*(x-xt)),adjointFilename)
    return
    
x = np.random.rand(dim)
forward(x)
adjoint(x,gradientFilename)

Line minimization: $N$-point brute-force search

In [4]:
NumSearch = 10
golden_ratio = 1.618034
initial_step = 0.1
tol = 1.0e-8

def mnbrak(x0,cg_direction):
    stepBracket = np.zeros(3)
    JBracket = np.zeros(3)
    from optimization import readScalar

    JBracket[0] = readScalar(forwardFilename)
    
    steps, Js = np.zeros(NumSearch+2), np.zeros(NumSearch+2)
    steps[1:NumSearch+1] = initial_step*golden_ratio**np.linspace(0.0,NumSearch-1,NumSearch)
    Js[0] = JBracket[0]

    while True:
        #The first and the last index will always be equal to outer bracket.
        for k in range(NumSearch):
            x1 = x0 + steps[k+1]*cg_direction
            forward(x1)
            Js[k+1] = readScalar(forwardFilename)
        if (steps[NumSearch+1]==0.0):
            steps[NumSearch+1], Js[NumSearch+1] = steps[NumSearch], Js[NumSearch]
        minidx = np.argmin(Js)

        if ( (minidx==0) | (Js[1]>=Js[0]) ):
            stepBracket[2], JBracket[2] = steps[1], Js[1]
            h = (stepBracket[2]-stepBracket[0])/(NumSearch+1)
            steps[1:] = np.linspace(stepBracket[0]+h,stepBracket[2],NumSearch+1)
            Js[-1:] = JBracket[2]
        elif (minidx==NumSearch+1):
            stepBracket[0], JBracket[0] = steps[minidx], Js[minidx]
            steps[:NumSearch+1] = stepBracket[0]*golden_ratio**np.linspace(0.0,NumSearch,NumSearch+1)
            steps[-1:] = 0.0
        else:
            stepBracket = steps[minidx-1:minidx+2]
            JBracket = Js[minidx-1:minidx+2]
            break
    return stepBracket, JBracket

In [7]:
grad = readVector(gradientFilename)

stepBracket, JBracket = mnbrak(x,-grad)
print (stepBracket)
print (JBracket)

[0.68541022 1.10901703 1.79442727]
[0.02235627 0.00268472 0.1425668 ]


In [18]:
def linmin(x0, stepBracket0, JBracket0, cg_direction, tol=1.0e-7):
    stepBracket, JBracket = stepBracket0, JBracket0
    bracketSize = (stepBracket[2]-stepBracket[0])/2.0/stepBracket[1]
    
    while (bracketSize>tol):
        steps, Js = np.zeros(NumSearch+3), np.zeros(NumSearch+3)
        steps[0], Js[0] = stepBracket[0], JBracket[0]
        steps[-1:], Js[-1:] = stepBracket[-1:], JBracket[-1:]
        h = (stepBracket[2]-stepBracket[0])/(NumSearch+1)
        intermediate_steps = np.linspace(stepBracket[0]+h,stepBracket[2]-h,NumSearch)
        intermediate_Js = np.zeros(NumSearch)
        idx, remainder = int((stepBracket[1]-stepBracket[0])//h), (stepBracket[1]-stepBracket[0])%h
        if (remainder==0.0):
            intermediate_steps[idx-1] -= 0.5*h
        steps[1:idx+1], steps[idx+1], steps[idx+2:NumSearch+2] = intermediate_steps[:idx],            \
                                                                 stepBracket[1],                      \
                                                                 intermediate_steps[idx:NumSearch]
        Js[idx+1] = JBracket[1]

        from optimization import readScalar
        for k in range(NumSearch):
            x1 = x0 + intermediate_steps[k]*cg_direction
            forward(x1)
            intermediate_Js[k] = readScalar(forwardFilename)
        Js[1:idx+1], Js[idx+2:NumSearch+2] = intermediate_Js[:idx], intermediate_Js[idx:NumSearch]
        minidx = np.argmin(Js)
        stepBracket, JBracket = steps[minidx-1:minidx+2], Js[minidx-1:minidx+2]
        bracketSize = (stepBracket[2]-stepBracket[0])/2.0/stepBracket[1]

    return stepBracket, JBracket
    
stepBracket, JBracket = linmin(x, stepBracket,JBracket,-grad)
print (stepBracket)
print (JBracket)

4 0.020327889906974983
5 0.009165430166709128
3 0.001666441848492648
5 0.00022724207024893417
7 2.0658370022670725e-05
5 5.63410091521237e-06
3 1.0243819845914066e-06
6 1.3968845249199474e-07
5 2.116491693016087e-08
[0.99999996 1.00000001 1.00000005]
[3.017082e-16 7.557166e-18 5.229367e-16]
